In [32]:
import pandas as pd
import json
import traceback
import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import datetime
from datetime import date
from datetime import timedelta
import io
df=pd.read_csv(f'D:/Python/NAVER/new_keyword/result_data/grp_new/keyword_crawling_20210113.txt',header=None, sep = "\t", error_bad_lines=False)
len(df)##9만줄이 맞음
def keyword_cleansing (i,json_txt):
    #print(i,json_txt)
    try  :
        json_txt=json.loads(json_txt)
        #print(json_txt)
        return json_txt[i]
    except : 
        traceback.print_exc()
        print(json_txt)
        return {'title':0,'data':0}
    
def yearmonth_calculator(date):  # 날짜를 넣으면 주차를 반환 (yyyymmdd)
    today_datetime = datetime.strptime(date, '%Y%m%d')  # w_date의 날짜
    today_iso = today_datetime.isoweekday()  # w_date의 요일 (월1 화2 수3 목4 금5 토6 일7)
    reference_date = today_datetime + timedelta(days=(4-today_iso))  # 주차를 정하는 기준날짜 (목요일의 날짜)
    reference_date_str = datetime.strftime(reference_date, '%Y%m%d')  # 기준날짜의 string
    return reference_date.strftime("%Y%m")
    #except : pass

In [33]:
result_df=pd.DataFrame()
group_id=0
for i in range(0,4) :
    #df_name = "df_"+str(i)
    
    tmp_df=pd.DataFrame()  
    tmp_df['keyword']=df[:-1].apply(lambda x: keyword_cleansing(i,x[0])['title'],axis = 1)
    tmp_df['data']=df[:-1].apply(lambda x: keyword_cleansing(i,x[0])['data'],axis = 1)
    tmp_df['group_id']=df[:-1].apply(lambda x: x.index)
    group_id +=1
    #tmp_df['cluster_id'] = df
    #print(tmp_df)
    result_df=pd.concat([result_df,tmp_df])
    #print(len(result_df))
#result_df['cluster_id'] = result_df.index

In [34]:
result_df.groupby('group_id').count()

,keyword,data
group_id,,
0,4,4
1,4,4
2,4,4
3,4,4
4,4,4
...,...,...
1069,4,4
1070,4,4
1071,4,4


In [35]:
try:
    conn = psycopg2.connect(host='10.123.184.193', dbname='edw', user='sys', password='eos40d')
    cur = conn.cursor(cursor_factory=RealDictCursor)
except Exception:
    print('Fail to connect DB')

sql = "select * from sbd.sbds_search_cnt_week where start_date = '20191104' and keyword = '아파트'"
cur.execute(sql)
result = cur.fetchall()
df2 = pd.DataFrame(result)
df2=df2.rename(columns={'start_date':'stand_start_date','scale':'stand_scale' })

In [36]:
merged_df = pd.merge(result_df,df2, on="keyword",how="left")
merged_df=merged_df.sort_values(['group_id','stand_scale'])
merged_df                                 
                                 

,keyword,data,group_id,stand_start_date,end_date,year_month,stand_scale,indate
0,아파트,"[{'period': '20170102', 'value': 50.86082}, {'...",0,2019-11-04,2019-11-10,201911,10624.0,2020-06-17
1074,아동가죽무스탕,"[{'period': '20181029', 'value': 0.03009}, {'p...",0,NaN,NaN,NaN,NaN,NaN
2148,아동경량구스자켓,[],0,NaN,NaN,NaN,NaN,NaN
3222,아동경량퀼팅롱자켓,[],0,NaN,NaN,NaN,NaN,NaN
1,아파트,"[{'period': '20170102', 'value': 50.86082}, {'...",1,2019-11-04,2019-11-10,201911,10624.0,2020-06-17
...,...,...,...,...,...,...,...,...
4294,베이비패딩트렌치코트,[],1072,NaN,NaN,NaN,NaN,NaN
1073,아파트,"[{'period': '20170102', 'value': 50.86082}, {'...",1073,2019-11-04,2019-11-10,201911,10624.0,2020-06-17
2147,주니어코트패딩,[],1073,NaN,NaN,NaN,NaN,NaN
3221,주니어패딩트렌치,[],1073,NaN,NaN,NaN,NaN,NaN


In [37]:
def caculator_2nd(tmp_df,standard_start_date,return_start_date,date_scale) :
    #print(tmp_df)
    if len(tmp_df) == 4:
        #tmp_df[date_scale].iloc[0] = 10624
        stand_scale = 10624
        stand_ratio=[k['value'] for k in tmp_df['data'].iloc[0] if k['period'] == standard_start_date]
        
        #print(stand_scale,ratio_1,stand_ratio)
        ratio_1 = [k['value'] for k in tmp_df['data'].iloc[1] if k['period'] == return_start_date]
        if len(ratio_1) ==1 : tmp_df[date_scale].iloc[1] = stand_scale*ratio_1[0]/stand_ratio[0] 
        else : tmp_df[date_scale].iloc[1] = 0
            
        ratio_2 = [k['value'] for k in tmp_df['data'].iloc[2] if k['period'] == return_start_date]
        if len(ratio_2) ==1 : tmp_df[date_scale].iloc[2] = stand_scale*ratio_2[0]/stand_ratio[0] 
        else : tmp_df[date_scale].iloc[2] = 0
            
        ratio_3 = [k['value'] for k in tmp_df['data'].iloc[3] if k['period'] == return_start_date]
        if len(ratio_3) ==1 : tmp_df[date_scale].iloc[3] = stand_scale*ratio_3[0]/stand_ratio[0] 
        else : tmp_df[date_scale].iloc[3] = 0            
            
    return tmp_df

In [42]:
#date_list=monday_list
date_list=['20210104']
for date in date_list : 
    #print(date)
    w_date = datetime.now().strftime("%Y%m%d")
    return_start_date_tmp = datetime.strptime(date, '%Y%m%d') ###수정
    return_start_date=datetime.strftime(return_start_date_tmp,'%Y%m%d')
    return_end_date = return_start_date_tmp + timedelta(days=6)
    return_end_date=datetime.strftime(return_end_date,'%Y%m%d')
    date_scale = return_start_date+"_scale"
    print(return_start_date,return_end_date)

    cluster_id_list = merged_df['group_id'].unique()
    stand_start_date = '20191104'


    merged_df[date_scale]=None
    #merged_df[merged_df['keyword']=='아파트'][date_scale]=10755
    final_df=pd.DataFrame()
    for i in range(0,len(cluster_id_list)) : 
        tmp_df=merged_df[merged_df['group_id']==i]
        tmp_df=caculator_2nd(tmp_df,stand_start_date,return_start_date,date_scale)
        final_df=pd.concat([final_df,tmp_df])

    final_df=final_df.rename(columns={date_scale:'scale'})
    droped_index=final_df[final_df['keyword']=='아파트'].index
    final_df=final_df.drop(droped_index)
    final_df['start_date']= return_start_date
    final_df['end_date']=return_end_date
    final_df['indate'] =w_date
    final_df['year_month'] = yearmonth_calculator(return_start_date)
    final_df=final_df.astype({'scale' : 'int'})
    final_df=final_df[['keyword','start_date','end_date','year_month','scale','indate']]

    ####db 업로드
    try:
        conn = psycopg2.connect(host='10.123.184.193', dbname='edw', user='sys', password='eos40d')
        cur = conn.cursor(cursor_factory=RealDictCursor)
    except Exception:
        print('Fail to connect DB')
    output = io.StringIO()
    # ignore the index


    tbl='sbd.sbds_search_cnt_week'
    final_df.to_csv(output, header=False, index=False)
    output.seek(0)
    cur.copy_expert("COPY %s FROM STDIN WITH CSV" % tbl, output)
    conn.commit()
    cur.close()

20210104 20210110


c:\users\jeong_jaekeun\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\users\jeong_jaekeun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\jeong_jaekeun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

In [39]:
from datetime import date, timedelta
monday_list=[]
yearmonday_list=[[2017,1,2],[2018,1,1],[2019,1,7],[2020,1,6]]
for yearmon in yearmonday_list:
    year = yearmon[0]
    date_object = date(year, yearmon[1], yearmon[2])
    date_object += timedelta(days=1-date_object.isoweekday())
    
    while date_object.year == year:
        #print(date_object.strftime("%Y%m%d"))
        monday_list.append(date_object.strftime("%Y%m%d"))
        date_object += timedelta(days=7)

In [40]:
monday_list

['20170102',
 '20170109',
 '20170116',
 '20170123',
 '20170130',
 '20170206',
 '20170213',
 '20170220',
 '20170227',
 '20170306',
 '20170313',
 '20170320',
 '20170327',
 '20170403',
 '20170410',
 '20170417',
 '20170424',
 '20170501',
 '20170508',
 '20170515',
 '20170522',
 '20170529',
 '20170605',
 '20170612',
 '20170619',
 '20170626',
 '20170703',
 '20170710',
 '20170717',
 '20170724',
 '20170731',
 '20170807',
 '20170814',
 '20170821',
 '20170828',
 '20170904',
 '20170911',
 '20170918',
 '20170925',
 '20171002',
 '20171009',
 '20171016',
 '20171023',
 '20171030',
 '20171106',
 '20171113',
 '20171120',
 '20171127',
 '20171204',
 '20171211',
 '20171218',
 '20171225',
 '20180101',
 '20180108',
 '20180115',
 '20180122',
 '20180129',
 '20180205',
 '20180212',
 '20180219',
 '20180226',
 '20180305',
 '20180312',
 '20180319',
 '20180326',
 '20180402',
 '20180409',
 '20180416',
 '20180423',
 '20180430',
 '20180507',
 '20180514',
 '20180521',
 '20180528',
 '20180604',
 '20180611',
 '20180618',

In [115]:
final_df

,keyword,data,cluster_id,stand_start_date,end_date,year_month,stand_scale,indate,20191111_scale
0,아파트,"[{'period': '20170102', 'value': 50.86082}, {'...",0,2019-11-04,2019-11-10,201911,10624.0,2020-06-17,10624
5,로고점퍼,"[{'period': '20170123', 'value': 0.03009}, {'p...",0,NaN,NaN,NaN,NaN,NaN,None
10,로고패딩,"[{'period': '20170109', 'value': 0.03611}, {'p...",0,NaN,NaN,NaN,NaN,NaN,None
15,논후드패딩,[],0,NaN,NaN,NaN,NaN,NaN,None
1,아파트,"[{'period': '20170102', 'value': 50.86082}, {'...",1,2019-11-04,2019-11-10,201911,10624.0,2020-06-17,10624
6,히든후드패딩,[],1,NaN,NaN,NaN,NaN,NaN,None
11,히든후드점퍼,[],1,NaN,NaN,NaN,NaN,NaN,None
16,리사이클점퍼,[],1,NaN,NaN,NaN,NaN,NaN,None
2,아파트,"[{'period': '20170102', 'value': 50.86082}, {'...",2,2019-11-04,2019-11-10,201911,10624.0,2020-06-17,10624
7,리사이클패딩,"[{'period': '20170102', 'value': 0.06621}, {'p...",2,NaN,NaN,NaN,NaN,NaN,20.7454


In [40]:
merged_df['data'].iloc[0]

[{'period': '20170102', 'value': 50.86082},
 {'period': '20170109', 'value': 60.23356},
 {'period': '20170116', 'value': 58.59619},
 {'period': '20170123', 'value': 55.03852},
 {'period': '20170130', 'value': 59.27642},
 {'period': '20170206', 'value': 56.04382},
 {'period': '20170213', 'value': 56.098},
 {'period': '20170220', 'value': 57.03708},
 {'period': '20170227', 'value': 53.43125},
 {'period': '20170306', 'value': 53.90079},
 {'period': '20170313', 'value': 54.99638},
 {'period': '20170320', 'value': 54.41247},
 {'period': '20170327', 'value': 51.30628},
 {'period': '20170403', 'value': 50.96917},
 {'period': '20170410', 'value': 50.00601},
 {'period': '20170417', 'value': 54.31615},
 {'period': '20170424', 'value': 46.81555},
 {'period': '20170501', 'value': 45.13002},
 {'period': '20170508', 'value': 50.93306},
 {'period': '20170515', 'value': 57.02504},
 {'period': '20170522', 'value': 63.49626},
 {'period': '20170529', 'value': 63.70695},
 {'period': '20170605', 'value': 6

In [34]:
result_df.sort_values('cluster_id')

,keyword,data,cluster_id
0,아파트,"[{'period': '20170102', 'value': 50.86082}, {'...",0
0,논후드점퍼,[],0
0,로고점퍼,"[{'period': '20170123', 'value': 0.03009}, {'p...",0
0,논후드패딩,[],0
0,로고패딩,"[{'period': '20170109', 'value': 0.03611}, {'p...",0
1,아파트,"[{'period': '20170102', 'value': 50.86082}, {'...",1
1,0,0,1
1,히든후드패딩,[],1
1,리사이클점퍼,[],1
1,히든후드점퍼,[],1
